# Sentiment Analysis

Here we'll train a sentiment analysis model to validate the data from the API.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import pandas as pd

In [3]:
sentiment140_path = Path('../datasets/sentiment140/sentiment140.csv')
data = pd.read_csv(sentiment140_path)

In [4]:
data.head()

,label,tweet
0,0,@whiskey_kitten www.Pandora.com - plays music ...
1,0,studying for a test I hope not to fail....most...
2,4,"@BlowhornOz Oh! Doesn't sound so good, I got t..."
3,0,tomorrow is my last day at A&amp;D HS fml and...
4,0,Journalism has no future? That sounds pretty m...


## Data preprocessing

Preprocess the texts:
- Convert to Lowercase: Convert all characters from the text to lowercase
- Remove special characters: Remove links and usernames and transform emojis to text
- Remove repetitions: Remove char repetitions (e.g. whaaaaaat => what)
- Remove Stop words: Remove common stop words

In [5]:
import re
from time import time
import nltk
from emoji import demojize

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rmohashi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
texts = data.tweet

start = time()
# Lowercasing
texts = texts.str.lower()

# Remove special chars
texts = texts.str.replace(r"(http|@)\S+", "")
texts = texts.apply(demojize)
texts = texts.str.replace(r"::", ": :")
texts = texts.str.replace(r"’", "'")
texts = texts.str.replace(r"[^a-z\':_]", " ")

# Remove repetitions
pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
texts = texts.str.replace(pattern, r"\1")

# Transform short negation form
texts = texts.str.replace(r"(can't|cannot)", 'can not')
texts = texts.str.replace(r"n't", ' not')

# Remove stop words
stopwords = nltk.corpus.stopwords.words('english')
stopwords.remove('not')
stopwords.remove('nor')
stopwords.remove('no')
texts = texts.apply(
    lambda x: ' '.join([word for word in x.split() if word not in stopwords])
)

print("Time to clean up: {:.2f} sec".format(time() - start))

data.tweet = texts

Time to clean up: 78.03 sec


## Tokenize

Transform the text corpus to a vector representation

- **num_words**: Number of words to use

In [7]:
num_words = 10000

In [8]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
tokenizer = Tokenizer(num_words=num_words, lower=True)
tokenizer.fit_on_texts(data.tweet)

file_to_save = Path('../datasets/sentiment140/tokenizer.pickle').resolve()
with file_to_save.open('wb') as file:
    pickle.dump(tokenizer, file)

## Split data

Split the dataset in train and validation data

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train = pd.DataFrame(columns=['label', 'tweet'])
validation = pd.DataFrame(columns=['label', 'tweet'])
for label in data.label.unique():
    label_data = data[data.label == label]
    train_data, validation_data = train_test_split(label_data, test_size=0.3)
    train = pd.concat([train, train_data])
    validation = pd.concat([validation, validation_data])

## Model

Define the Bidirectional GRU model

In [12]:
from tensorflow.keras.layers import Input, Embedding, GRU
from tensorflow.keras.layers import Dropout, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Dense
from tensorflow.keras.models import Sequential

In [13]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
embedding_dim = 200
input_length = 100
gru_units = 128
gru_dropout = 0.1
recurrent_dropout = 0.1
dropout = 0.1

In [14]:
model = Sequential()
model.add(Embedding(
    input_dim=input_dim,
    output_dim=embedding_dim,
    input_shape=(input_length,)
))

model.add(Bidirectional(GRU(
    gru_units,
    return_sequences=True,
    dropout=gru_dropout,
    recurrent_dropout=recurrent_dropout
)))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(1, activation='sigmoid'))

W0716 13:36:20.397812 140315330369344 deprecation.py:506] From /home/rmohashi/anaconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0716 13:36:20.410246 140315330369344 deprecation.py:506] From /home/rmohashi/anaconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0716 13:36:20.413324 140315330369344 deprecation.py:506] From /home/rmohashi/anaconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/op

In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W0716 13:36:20.902724 140315330369344 deprecation.py:323] From /home/rmohashi/anaconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 200)          2000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 256)          252672    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,260,929
Trainable params: 2,260,929
Non-trainable params: 0
______________________________________________

## Prepare the data

Prepare the model input data

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
train_sequences = [text.split() for text in train.tweet]
validation_sequences = [text.split() for text in validation.tweet]
list_tokenized_train = tokenizer.texts_to_sequences(train_sequences)
list_tokenized_validation = tokenizer.texts_to_sequences(validation_sequences)

x_train = pad_sequences(list_tokenized_train, maxlen=input_length)
x_validation = pad_sequences(list_tokenized_validation, maxlen=input_length)
y_train = train.label.replace(4, 1)
y_validation = validation.label.replace(4, 1)

## Train model

Do the training process with the given data

In [18]:
batch_size = 128
epochs = 1

In [19]:
model.fit(
    x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_validation, y_validation),
)

Train on 280000 samples, validate on 120000 samples
280000/280000 [==============================] - 374s 1ms/sample - loss: 0.4637 - acc: 0.7804 - val_loss: 0.4366 - val_acc: 0.7937


In [20]:
model_file = Path('../models/sentiment_analysis/gru_model.h5').resolve()
model.save_weights(model_file.as_posix())